---

_You are currently looking at **version 1.0** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-machine-learning/resources/bANLa) course resource._

---

## Understanding and Predicting Property Maintenance Fines

This is based on a data challenge from the Michigan Data Science Team ([MDST](http://midas.umich.edu/mdst/)). 

The Michigan Data Science Team ([MDST](http://midas.umich.edu/mdst/)) and the Michigan Student Symposium for Interdisciplinary Statistical Sciences ([MSSISS](https://sites.lsa.umich.edu/mssiss/)) have partnered with the City of Detroit to help solve one of the most pressing problems facing Detroit - blight. [Blight violations](http://www.detroitmi.gov/How-Do-I/Report/Blight-Complaint-FAQs) are issued by the city to individuals who allow their properties to remain in a deteriorated condition. Every year, the city of Detroit issues millions of dollars in fines to residents and every year, many of these fines remain unpaid. Enforcing unpaid blight fines is a costly and tedious process, so the city wants to know: how can we increase blight ticket compliance?

The first step in answering this question is understanding when and why a resident might fail to comply with a blight ticket. This is where predictive modeling comes in. Your task is to predict whether a given blight ticket will be paid on time.

All data has been provided to us through the [Detroit Open Data Portal](https://data.detroitmi.gov/). **Only the data already included in your Coursera directory can be used for training the model.** Nonetheless, we encourage you to look into data from other Detroit datasets to help inform feature creation and model selection. We recommend taking a look at the following related datasets:

* [Building Permits](https://data.detroitmi.gov/Property-Parcels/Building-Permits/xw2a-a7tf)
* [Trades Permits](https://data.detroitmi.gov/Property-Parcels/Trades-Permits/635b-dsgv)
* [Improve Detroit: Submitted Issues](https://data.detroitmi.gov/Government/Improve-Detroit-Submitted-Issues/fwz3-w3yn)
* [DPD: Citizen Complaints](https://data.detroitmi.gov/Public-Safety/DPD-Citizen-Complaints-2016/kahe-efs3)
* [Parcel Map](https://data.detroitmi.gov/Property-Parcels/Parcel-Map/fxkw-udwf)

___

We provide you with two data files for use in training and validating your models: train.csv and test.csv. Each row in these two files corresponds to a single blight ticket, and includes information about when, why, and to whom each ticket was issued. The target variable is compliance, which is True if the ticket was paid early, on time, or within one month of the hearing data, False if the ticket was paid after the hearing date or not at all, and Null if the violator was found not responsible. Compliance, as well as a handful of other variables that will not be available at test-time, are only included in train.csv.

Note: All tickets where the violators were found not responsible are not considered during evaluation. They are included in the training set as an additional source of data for visualization, and to enable unsupervised and semi-supervised approaches. However, they are not included in the test set.

<br>

**File descriptions** (Use only this data for training your model!)

    train.csv - the training set (all tickets issued 2004-2011)
    test.csv - the test set (all tickets issued 2012-2016)
    addresses.csv & latlons.csv - mapping from ticket id to addresses, and from addresses to lat/lon coordinates. 
     Note: misspelled addresses may be incorrectly geolocated.

<br>

**Data fields**

train.csv & test.csv

    ticket_id - unique identifier for tickets
    agency_name - Agency that issued the ticket
    inspector_name - Name of inspector that issued the ticket
    violator_name - Name of the person/organization that the ticket was issued to
    violation_street_number, violation_street_name, violation_zip_code - Address where the violation occurred
    mailing_address_str_number, mailing_address_str_name, city, state, zip_code, non_us_str_code, country - Mailing address of the violator
    ticket_issued_date - Date and time the ticket was issued
    hearing_date - Date and time the violator's hearing was scheduled
    violation_code, violation_description - Type of violation
    disposition - Judgment and judgement type
    fine_amount - Violation fine amount, excluding fees
    admin_fee - $20 fee assigned to responsible judgments
state_fee - $10 fee assigned to responsible judgments
    late_fee - 10% fee assigned to responsible judgments
    discount_amount - discount applied, if any
    clean_up_cost - DPW clean-up or graffiti removal cost
    judgment_amount - Sum of all fines and fees
    grafitti_status - Flag for graffiti violations
    
train.csv only

    payment_amount - Amount paid, if any
    payment_date - Date payment was made, if it was received
    payment_status - Current payment status as of Feb 1 2017
    balance_due - Fines and fees still owed
    collection_status - Flag for payments in collections
    compliance [target variable for prediction] 
     Null = Not responsible
     0 = Responsible, non-compliant
     1 = Responsible, compliant
    compliance_detail - More information on why each ticket was marked compliant or non-compliant


___

## Evaluation

Your predictions will be given as the probability that the corresponding blight ticket will be paid on time.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC). 

Your grade will be based on the AUC score computed for your classifier. A model which with an AUROC of 0.7 passes this assignment, over 0.75 will recieve full points.
___

For this assignment, create a function that trains a model to predict blight ticket compliance in Detroit using `train.csv`. Using this model, return a series of length 61001 with the data being the probability that each corresponding ticket from `test.csv` will be paid, and the index being the ticket_id.

Example:

    ticket_id
       284932    0.531842
       285362    0.401958
       285361    0.105928
       285338    0.018572
                 ...
       376499    0.208567
       376500    0.818759
       369851    0.018528
       Name: compliance, dtype: float32

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
testdata = pd.read_csv("test.csv",encoding="cp1252",parse_dates = ['ticket_issued_date', 'hearing_date'])

In [3]:
traindata = pd.read_csv('train.csv',encoding="cp1252",parse_dates = ['ticket_issued_date', 'hearing_date'])

c:\program files\python35\lib\site-packages\IPython\core\interactiveshell.py:2698: DtypeWarning: Columns (11,12,31) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
traindata.dropna(how = 'all',axis = 1,inplace = True)

In [5]:

traindata.memory_usage().sum()

66080864

It can be made more efficient by reducing un-necessary memory usage.This can be done grpouping similar things into categories

In [6]:
# reduce memory usage by casting
for i in range(len(traindata.columns)):
    if len(traindata[traindata.columns[i]].unique()) < 250:
        traindata[traindata.columns[i]] = traindata[traindata.columns[i]] .astype('category')
        

In [7]:
traindata.memory_usage().sum()

33321338

More than half memory is used saved!!

Now lets drop all things with Null values in it

In [8]:
#drop all with nan
traindata = traindata.dropna(axis = 1,how  = 'all')
traindata = traindata.dropna(axis = 0,how  = 'all')

We can see that many fields have many missing datas so let's get rid of columns  with more than 30 % missing data

In [9]:
#missing data handle
ttlnull = traindata.isnull().sum().sort_values(ascending = False)
per = (traindata.isnull().sum()/traindata.isnull().count()).sort_values(ascending = False)
missing_data = pd.concat([ttlnull, per], axis=1, keys = ['Total', 'Percent'])
missing_data.head(10)

,Total,Percent
grafitti_status,250305,0.999996
non_us_str_code,250303,0.999988
collection_status,213409,0.852592
payment_date,209193,0.835749
compliance,90426,0.361262
hearing_date,12491,0.049903
mailing_address_str_number,3602,0.014390
state,93,0.000372
violator_name,34,0.000136
mailing_address_str_name,4,0.000016


In [10]:

traindata.drop(missing_data[missing_data['Percent']>0.5].index,axis = 1,inplace = True)
#drop same columns from test data too
missing_data[missing_data['Percent']>0.5].index

Index(['grafitti_status', 'non_us_str_code', 'collection_status',
       'payment_date'],
      dtype='object')

Above columns are worthless as more than 50% of time they are NaN so they are not good for predicting

Now lets delete the columns which have all values same

In [11]:
#remove all columns of equal values...

length = len(traindata.columns)
for i in range (length):
    if len(traindata[traindata.columns[i]].unique()) == 1:
         print(traindata.columns[i])
#         
#         length = length -1


clean_up_cost


Only 'clean_up_cost' has all the values same (i.e = 0) so it is also independent of our target variable as 

In [12]:
traindata.drop(labels = 'clean_up_cost' ,axis = 1,inplace = True)
testdata.drop(labels = 'clean_up_cost' ,axis = 1,inplace = True)



Let's see how many more null columns are there

In [13]:
traindata.isnull().sum()

ticket_id                         0
agency_name                       0
inspector_name                    0
violator_name                    34
violation_street_number           0
violation_street_name             0
mailing_address_str_number     3602
mailing_address_str_name          4
city                              0
state                            93
zip_code                          1
country                           0
ticket_issued_date                0
hearing_date                  12491
violation_code                    0
violation_description             0
disposition                       0
fine_amount                       1
admin_fee                         0
state_fee                         0
late_fee                          0
discount_amount                   0
judgment_amount                   0
payment_amount                    0
balance_due                       0
payment_status                    0
compliance_detail                 0
compliance                  

street number , street name can be null as people have different format each time but are they relevant??

In [14]:
address = pd.read_csv('addresses.csv')
latlons = pd.read_csv('latlons.csv')

In [15]:
address = address.set_index('address').join(latlons.set_index('address'), how='left')
address.head()

,ticket_id,lat,lon
address,,,
"-11064 gratiot, Detroit MI",328722,42.406935,-82.995599
"-11871 wilfred, Detroit MI",350971,42.411288,-82.993674
"-15126 harper, Detroit MI",344821,42.406402,-82.957525
"0 10th st, Detroit MI",24928,42.325689,-83.064330
"0 10th st, Detroit MI",71887,42.325689,-83.064330


In [16]:
traindata = traindata.set_index('ticket_id').join(address.set_index('ticket_id'))
testdata = testdata.set_index('ticket_id').join(address.set_index('ticket_id'))

In [17]:
traindata.columns

Index(['agency_name', 'inspector_name', 'violator_name',
       'violation_street_number', 'violation_street_name',
       'mailing_address_str_number', 'mailing_address_str_name', 'city',
       'state', 'zip_code', 'country', 'ticket_issued_date', 'hearing_date',
       'violation_code', 'violation_description', 'disposition', 'fine_amount',
       'admin_fee', 'state_fee', 'late_fee', 'discount_amount',
       'judgment_amount', 'payment_amount', 'balance_due', 'payment_status',
       'compliance_detail', 'compliance', 'lat', 'lon'],
      dtype='object')

Now List of Items I think are irrevant..... As all this info should be covered in lan and log are below

In [18]:
list_to_remove = [ 'violator_name',
       'violation_street_number', 'violation_street_name',
       'mailing_address_str_number', 'mailing_address_str_name', 'city',
       'state', 'zip_code', 'country']

In [19]:
traindata.drop(list_to_remove,axis = 1,inplace = True)
testdata.drop(list_to_remove,axis = 1,inplace = True)

In [20]:
testdata.head(3)

,agency_name,inspector_name,violation_zip_code,non_us_str_code,ticket_issued_date,hearing_date,violation_code,violation_description,disposition,fine_amount,admin_fee,state_fee,late_fee,discount_amount,judgment_amount,grafitti_status,lat,lon
ticket_id,,,,,,,,,,,,,,,,,,
284932,Department of Public Works,"Granberry, Aisha B",NaN,NaN,2012-01-04 14:00:00,2012-01-19 09:00:00,22-2-61,Failure to secure City or Private solid waste ...,Responsible by Default,200.0,20.0,10.0,20.0,0.0,250.0,NaN,42.407581,-82.986642
285362,Department of Public Works,"Lusk, Gertrina",NaN,NaN,2012-01-05 09:50:00,2012-02-06 09:00:00,22-2-88(b),Allowing bulk solid waste to lie or accumulate...,Responsible by Default,1000.0,20.0,10.0,100.0,0.0,1130.0,NaN,42.426239,-83.238259
285361,Department of Public Works,"Lusk, Gertrina",NaN,NaN,2012-01-05 09:50:00,2012-02-06 09:00:00,22-2-43,Improper placement of Courville container betw...,Responsible by Default,100.0,20.0,10.0,10.0,0.0,140.0,NaN,42.426239,-83.238259


Let's reduce more feature,we can just use Total amount to pay instead of below features 

In [21]:
traindata['Total_amount_to_pay'] = traindata['fine_amount']+ traindata['admin_fee']+traindata['state_fee']+traindata['late_fee']-traindata['discount_amount']
testdata['Total_amount_to_pay'] = testdata['fine_amount']+ testdata['admin_fee']+testdata['state_fee']+testdata['late_fee']-testdata['discount_amount']

In [22]:
list_drop = ['fine_amount','admin_fee','state_fee','late_fee','discount_amount']
traindata.drop(list_drop,axis = 1,inplace = True)
testdata.drop(list_drop,axis = 1,inplace = True)

In [23]:
traindata.columns

Index(['agency_name', 'inspector_name', 'ticket_issued_date', 'hearing_date',
       'violation_code', 'violation_description', 'disposition',
       'judgment_amount', 'payment_amount', 'balance_due', 'payment_status',
       'compliance_detail', 'compliance', 'lat', 'lon', 'Total_amount_to_pay'],
      dtype='object')

In [24]:
testdata.columns

Index(['agency_name', 'inspector_name', 'violation_zip_code',
       'non_us_str_code', 'ticket_issued_date', 'hearing_date',
       'violation_code', 'violation_description', 'disposition',
       'judgment_amount', 'grafitti_status', 'lat', 'lon',
       'Total_amount_to_pay'],
      dtype='object')

Now lets fill NaN values of above calculated fields to train the model,As they are very less compared to original dataset we can 
drop them too,but dropping them from testSet isn't allowed so let's fill them

In [25]:
traindata.dropna(subset = ['lat','lon','Total_amount_to_pay'],inplace = True)
testdata['lat'].fillna(testdata.lat.mean(),inplace = True)
testdata['lon'].fillna(testdata.lon.mean(),inplace = True)
testdata.isnull().sum()

agency_name                  0
inspector_name               0
violation_zip_code       36977
non_us_str_code          61001
ticket_issued_date           0
hearing_date              2197
violation_code               0
violation_description        0
disposition                  0
judgment_amount              0
grafitti_status          58780
lat                          0
lon                          0
Total_amount_to_pay          0
dtype: int64

Now time_gap between dates is only needed so let's calculate that

In [26]:
#traindata = traindata[~traindata['hearing_date'].isnull()]
traindata['time_gap'] = traindata['hearing_date']-traindata['ticket_issued_date']
traindata['time_gap'] = traindata['time_gap'].dt.days
traindata['time_gap'].fillna(73,inplace = True)
traindata.drop(['hearing_date','ticket_issued_date'],axis = 1,inplace = True)

testdata['time_gap'] = testdata['hearing_date']-testdata['ticket_issued_date']
testdata['time_gap'] = testdata['time_gap'].dt.days
testdata['time_gap'].fillna(73,inplace = True)
testdata.drop(['hearing_date','ticket_issued_date'],axis = 1,inplace = True)

Now dropping features that aren't important and creating features from string categories

In [27]:
traindata.drop(['violation_description','payment_amount','balance_due','payment_status','violation_code','compliance_detail','inspector_name'],axis = 1,inplace = True)

string_features = ['disposition','agency_name']
traindata =  pd.get_dummies(traindata,columns = string_features,drop_first = True)
testdata =  pd.get_dummies(testdata,columns = string_features,drop_first = True)

Taking only NON NaN values for prediction

In [28]:
traindata = traindata[(traindata['compliance'] == 0) | (traindata['compliance'] == 1)]

In [29]:
traindata.head()

,judgment_amount,compliance,lat,lon,Total_amount_to_pay,time_gap,disposition_Not responsible by Determination,disposition_Not responsible by Dismissal,disposition_PENDING JUDGMENT,disposition_Responsible (Fine Waived) by Deter,disposition_Responsible by Admission,disposition_Responsible by Default,disposition_Responsible by Determination,disposition_SET-ASIDE (PENDING JUDGMENT),agency_name_Department of Public Works,agency_name_Detroit Police Department,agency_name_Health Department,agency_name_Neighborhood City Halls
ticket_id,,,,,,,,,,,,,,,,,,
22056,305.0,0.0,42.390729,-83.124268,305.0,369.0,0,0,0,0,0,1,0,0,0,0,0,0
27586,855.0,1.0,42.326937,-83.135118,855.0,378.0,0,0,0,0,0,0,1,0,0,0,0,0
22046,305.0,0.0,42.145257,-83.208233,305.0,323.0,0,0,0,0,0,1,0,0,0,0,0,0
18738,855.0,0.0,42.433466,-83.023493,855.0,253.0,0,0,0,0,0,1,0,0,0,0,0,0
18735,140.0,0.0,42.388641,-83.037858,140.0,251.0,0,0,0,0,0,1,0,0,0,0,0,0


In [30]:
traindata.columns


Index(['judgment_amount', 'compliance', 'lat', 'lon', 'Total_amount_to_pay',
       'time_gap', 'disposition_Not responsible by Determination',
       'disposition_Not responsible by Dismissal',
       'disposition_PENDING JUDGMENT',
       'disposition_Responsible (Fine Waived) by Deter',
       'disposition_Responsible by Admission',
       'disposition_Responsible by Default',
       'disposition_Responsible by Determination',
       'disposition_SET-ASIDE (PENDING JUDGMENT)',
       'agency_name_Department of Public Works',
       'agency_name_Detroit Police Department',
       'agency_name_Health Department', 'agency_name_Neighborhood City Halls'],
      dtype='object')

In [31]:
y = traindata['compliance']
X = traindata.drop('compliance',axis = 1)

Before submitting it to autograder let's test which model is actully better
I am reducing dataset to around 50k (to increse speed) from training data (as we have correct values here)

In [32]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [33]:
#just to test fast we are using very small dataset around 50K 1/5th of original
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size = 0.4)
X_train, X_test, y_train, y_test = train_test_split(X_test,y_test)

This is a the Base classifier any model shouldnot do worse than this

In [34]:
from sklearn.dummy import DummyClassifier
from sklearn.metrics import roc_auc_score
dummy_clf = DummyClassifier(strategy = 'most_frequent').fit(X_train,y_train)
ypred = dummy_clf.predict(X_test)
dummy_clf.score(X_test,y_test),dummy_clf.score(X_train,y_train),roc_auc_score(y_test,ypred)

(0.92894671003252438, 0.92775831873905434, 0.5)

In [35]:
from sklearn.naive_bayes import GaussianNB

nbclf = GaussianNB().fit(X_train, y_train)
y_pred = nbclf.predict(X_test)

roc_auc_score(y_test,y_pred),nbclf.score(X_test, y_test),(nbclf.score(X_train, y_train))

(0.7226458332543062, 0.85776832624468347, 0.85639229422066554)

Now let's try other classifiers to choose the best one for actual dataset

In [36]:
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

svm = SVC(random_state =0).fit(X_train_scaled,y_train)
ypred = svm.predict(X_test_scaled)
svm.score(X_test_scaled,y_test),svm.score(X_train_scaled,y_train),roc_auc_score(y_test,ypred)

(0.93201150863147364, 0.93163622717037775, 0.52685108089961807)

SVM takes a lot of time

In [37]:
from sklearn.neural_network import MLPRegressor
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()

X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

reg = MLPRegressor(hidden_layer_sizes=(20,20),random_state = 0).fit(X=X_train_scaled,y = y_train)
ypred = reg.predict(X_test_scaled)
reg.score(X_test_scaled,y_test),reg.score(X_train_scaled,y_train),roc_auc_score(y_test,ypred)

(0.13832227086072413, 0.13379956028942919, 0.76902951255201557)

In [38]:
from sklearn.linear_model import LinearRegression
linreg = LinearRegression().fit(X_train,y_train)
ypred = linreg.predict(X_test)
linreg.score(X_test,y_test),linreg.score(X_train,y_train),roc_auc_score(y_test,ypred)

(0.139432619079358, 0.13842039385234217, 0.77176584198836973)

In [39]:
from sklearn.ensemble import RandomForestRegressor
reg = RandomForestRegressor(max_depth = 6,random_state=0).fit(X_train, y_train)
ypred = reg.predict(X_test)
reg.score(X_test,y_test),reg.score(X_train,y_train),roc_auc_score(y_test,ypred)

(0.1753487994527938, 0.20402804659872675, 0.79495805207626047)

So From above analysis ,we can use randomforest or GaussianNB for best results,any classifier except SVMs should work
maybe svm requires more tuning or maybe more data.
I will be using RandomForest for my actual submission

Actual code actually has slight modification due to grader issues,But concepts used are same
Below code passes the grader with 100 points

In [40]:
import pandas as pd
import numpy as np

def blight_model():
    testdata = pd.read_csv('test.csv',encoding="cp1252")
    traindata = pd.read_csv('train.csv',encoding="cp1252")
    #drop if all are NaN

    traindata = traindata.dropna(axis = 1,how  = 'all')
    traindata = traindata.dropna(axis = 0,how  = 'all')
    

    
#     #missing data handle
#     ttlnull = traindata.isnull().sum().sort_values(ascending = False)
#     per = (traindata.isnull().sum()/traindata.isnull().count()).sort_values(ascending = False)
#     missing_data = pd.concat([ttlnull, per], axis=1, keys = ['Total', 'Percent'])
#     #missing_data
    
#     traindata.drop(missing_data[missing_data['Percent']>0.5].index,axis = 1,inplace = True)
#     traindata.drop(labels = 'clean_up_cost' ,axis = 1,inplace = True)  # as all values in this is zeroes
    
    #Do somethings about addresses
    address = pd.read_csv('addresses.csv')
    latlons = pd.read_csv('latlons.csv')
    address = address.set_index('address').join(latlons.set_index('address'), how='left')
    traindata = traindata.set_index('ticket_id').join(address.set_index('ticket_id'))
    testdata =  testdata.set_index('ticket_id').join(address.set_index('ticket_id'))
    
    
    list_to_remove = [ 'violator_name',
       'violation_street_number', 'violation_street_name',
       'mailing_address_str_number', 'mailing_address_str_name', 'city',
       'state', 'zip_code', 'country','grafitti_status','non_us_str_code','violation_description',
        'inspector_name','clean_up_cost','violation_code']
    
    list_rm_trn = ['collection_status','payment_date','payment_amount','payment_status',
                   'balance_due','compliance_detail']
    
    traindata.drop(list_to_remove,axis = 1,inplace = True)
    testdata.drop(list_to_remove,axis = 1,inplace = True)
    traindata.drop(list_rm_trn,axis = 1,inplace = True)
    testdata.drop('violation_zip_code',axis = 1,inplace = True)
    
    #reduce some features
    traindata['Total_amount_to_pay'] = traindata['fine_amount']+ traindata['admin_fee']+traindata['state_fee']+traindata['late_fee']-traindata['discount_amount']
    list_drop = ['fine_amount','admin_fee','state_fee','late_fee','discount_amount']
    traindata.drop(list_drop,axis = 1,inplace = True)
    
    testdata['Total_amount_to_pay'] = testdata['fine_amount']+ testdata['admin_fee']+testdata['state_fee']+testdata['late_fee']-testdata['discount_amount']
    testdata.drop(list_drop,axis = 1,inplace = True)
    
    #drop Nan Values while trainning
    traindata.dropna(subset = ['lat','lon','Total_amount_to_pay'],inplace = True) 
    testdata['lat'].fillna(testdata.lat.mean(),inplace = True)
    testdata['lon'].fillna(testdata.lon.mean(),inplace = True)
    ######### testdata['lat','lon','Total_amount_to_pay'] =  testdata['lat','lon','Total_amount_to_pay'].fillna
    
    #use new feature time_gap instead of 2 dates
    from datetime import date
    traindata['hearing_date'] = pd.to_datetime(traindata['hearing_date'])
    traindata['ticket_issued_date'] = pd.to_datetime(traindata['ticket_issued_date'])
    testdata['hearing_date'] = pd.to_datetime(testdata['hearing_date'])
    testdata['ticket_issued_date'] = pd.to_datetime(testdata['ticket_issued_date'])
    
    traindata['hearing_date'].fillna(method = 'pad',inplace = True)
    traindata['ticket_issued_date'].fillna(method = 'pad',inplace = True)
    testdata['hearing_date'].fillna(method = 'pad',inplace = True)
    testdata['ticket_issued_date'].fillna(method = 'pad',inplace = True)
    
    traindata['time_gap'] = traindata['hearing_date'].subtract(traindata['ticket_issued_date'])
    traindata['time_gap'] = traindata['time_gap'].dt.days
    traindata['time_gap'].fillna(73,inplace = True)
    traindata.drop(['hearing_date','ticket_issued_date'],axis = 1,inplace = True)
    
    testdata['time_gap'] = testdata['hearing_date'].subtract(testdata['ticket_issued_date'])
    testdata['time_gap'] = testdata['time_gap'].dt.days
    testdata['time_gap'].fillna(73,inplace = True)
    testdata.drop(['hearing_date','ticket_issued_date'],axis = 1,inplace = True)
    
    # reduce memory usage by casting
    for i in range(len(traindata.columns)):
        if len(traindata[traindata.columns[i]].unique()) < 250:
            traindata[traindata.columns[i]] = traindata[traindata.columns[i]] .astype('category')

    #using categorial string as features
    string_features = ['disposition','agency_name']
    traindata =  pd.get_dummies(traindata,columns = string_features,drop_first = True)
    testdata =  pd.get_dummies(testdata,columns = string_features,drop_first = True)

    #For training take good values only
    traindata = traindata[(traindata['compliance'] == 0) | (traindata['compliance'] == 1)]
    
    
    y_train = traindata['compliance']
    X_train = traindata.drop('compliance',axis = 1)
    X_test = testdata
    
    train_features = X_train
    train_features_set = set(train_features)
    
    for feature in set(train_features):
        if feature not in testdata:
            train_features_set.remove(feature)
    train_features = list(train_features_set)
    
    X_train = X_train[train_features]
    X_test =  X_test[train_features]
    
    from sklearn.ensemble import RandomForestRegressor
    reg = RandomForestRegressor(max_depth = 6).fit(X_train, y_train)
    y_pred = reg.predict(X_test)

    test_df = pd.read_csv('test.csv', encoding = "ISO-8859-1")
    test_df['compliance'] = y_pred
    test_df.set_index('ticket_id', inplace=True)
    
    
    return test_df.compliance

In [41]:
blight_model()

c:\program files\python35\lib\site-packages\IPython\core\interactiveshell.py:2808: DtypeWarning: Columns (11,12,31) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


ticket_id
284932    0.060068
285362    0.012516
285361    0.067258
285338    0.077760
285346    0.082422
285345    0.077760
285347    0.082422
285342    0.454940
285530    0.033876
284989    0.024854
285344    0.082422
285343    0.027466
285340    0.027466
285341    0.082422
285349    0.082422
285348    0.077760
284991    0.024854
285532    0.034823
285406    0.019184
285001    0.053731
285006    0.052784
285405    0.012516
285337    0.020096
285496    0.067258
285497    0.062595
285378    0.012516
285589    0.019184
285585    0.062595
285501    0.067258
285581    0.012516
            ...   
376367    0.019184
376366    0.030526
376362    0.215380
376363    0.454940
376365    0.019184
376364    0.030526
376228    0.035194
376265    0.034016
376286    0.109097
376320    0.031704
376314    0.031704
376327    0.273646
376385    0.273646
376435    0.331466
376370    0.454940
376434    0.067258
376459    0.067258
376478    0.012775
376473    0.035194
376484    0.023522
376482    0.021267
37

I have GuassionNB too for submission,It also gives full marks i.e AUC score of more than 75%

Some steps of tuning for some model isn't showned in this Notebook but they are very important,like Neural Networks without scaling was giving AUC score of around 0.53 only 

Models can be made more efficient too but This passes the test so I didn't try 